In [4]:
import pickle

enron_data = pickle.load(open("final_project/final_project_dataset.pkl", "r"))

In [5]:
print(enron_data["SKILLING JEFFREY K"])

{'salary': 1111258, 'to_messages': 3627, 'deferral_payments': 'NaN', 'total_payments': 8682716, 'exercised_stock_options': 19250000, 'bonus': 5600000, 'restricted_stock': 6843672, 'shared_receipt_with_poi': 2042, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 26093672, 'expenses': 29336, 'loan_advances': 'NaN', 'from_messages': 108, 'other': 22122, 'from_this_person_to_poi': 30, 'poi': True, 'director_fees': 'NaN', 'deferred_income': 'NaN', 'long_term_incentive': 1920000, 'email_address': 'jeff.skilling@enron.com', 'from_poi_to_this_person': 88}


In [7]:
print(len(enron_data))

146


In [10]:
print(len(enron_data["SKILLING JEFFREY K"]))

21


In [11]:
print(type(enron_data))

<type 'dict'>


In [23]:
pois = 0
for name, data in enron_data.items():
    if data['poi'] == True:
        pois += 1
print(pois)

18


In [26]:
print(enron_data['PRENTICE JAMES']['total_stock_value'])

1095040


In [27]:
print(enron_data['COLWELL WESLEY'])

{'salary': 288542, 'to_messages': 1758, 'deferral_payments': 27610, 'total_payments': 1490344, 'exercised_stock_options': 'NaN', 'bonus': 1200000, 'restricted_stock': 698242, 'shared_receipt_with_poi': 1132, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 698242, 'expenses': 16514, 'loan_advances': 'NaN', 'from_messages': 40, 'other': 101740, 'from_this_person_to_poi': 11, 'poi': True, 'director_fees': 'NaN', 'deferred_income': -144062, 'long_term_incentive': 'NaN', 'email_address': 'wes.colwell@enron.com', 'from_poi_to_this_person': 240}


In [28]:
print(enron_data["SKILLING JEFFREY K"]["exercised_stock_options"])

19250000


In [33]:
print(enron_data["FASTOW ANDREW S"]["total_payments"])
print(enron_data["SKILLING JEFFREY K"]["total_payments"])
print(enron_data["LAY KENNETH L"]["total_payments"])

2424083
8682716
103559793


In [30]:
print(enron_data.keys())

['METTS MARK', 'BAXTER JOHN C', 'ELLIOTT STEVEN', 'CORDES WILLIAM R', 'HANNON KEVIN P', 'MORDAUNT KRISTINA M', 'MEYER ROCKFORD G', 'MCMAHON JEFFREY', 'HORTON STANLEY C', 'PIPER GREGORY F', 'HUMPHREY GENE E', 'UMANOFF ADAM S', 'BLACHMAN JEREMY M', 'SUNDE MARTIN', 'GIBBS DANA R', 'LOWRY CHARLES P', 'COLWELL WESLEY', 'MULLER MARK S', 'JACKSON CHARLENE R', 'WESTFAHL RICHARD K', 'WALTERS GARETH W', 'WALLS JR ROBERT H', 'KITCHEN LOUISE', 'CHAN RONNIE', 'BELFER ROBERT', 'SHANKMAN JEFFREY A', 'WODRASKA JOHN', 'BERGSIEKER RICHARD P', 'URQUHART JOHN A', 'BIBI PHILIPPE A', 'RIEKER PAULA H', 'WHALEY DAVID A', 'BECK SALLY W', 'HAUG DAVID L', 'ECHOLS JOHN B', 'MENDELSOHN JOHN', 'HICKERSON GARY J', 'CLINE KENNETH W', 'LEWIS RICHARD', 'HAYES ROBERT E', 'MCCARTY DANNY J', 'KOPPER MICHAEL J', 'LEFF DANIEL P', 'LAVORATO JOHN J', 'BERBERIAN DAVID', 'DETMERING TIMOTHY J', 'WAKEHAM JOHN', 'POWERS WILLIAM', 'GOLD JOSEPH', 'BANNANTINE JAMES M', 'DUNCAN JOHN H', 'SHAPIRO RICHARD S', 'SHERRIFF JOHN R', 'SHELBY 

In [36]:
with_salary = 0
with_email = 0

for name, data in enron_data.items():
    if data['salary'] != 'NaN':
        with_salary += 1
    if data['email_address'] != 'NaN':
        with_email += 1

print(with_salary)
print(with_email)

95
111


In [57]:
with_payments = 0
for name, data in enron_data.items():
    if data['total_payments'] == 'NaN':
        with_payments += 1

print(with_payments)
print(with_payments * 100 / len(enron_data))

21
14


In [58]:
pois = 0
no_payments = 0
for name, data in enron_data.items():
    if data['poi'] == True:
        pois += 1
        if data['total_payments'] == 'NaN':
            no_payments += 1

print(no_payments)
print(no_payments * 100 / pois)
print(no_payments * 100 / (pois + 10))
print(pois)

0
0
0
18


In [55]:
print(len(enron_data) + 10)

156


In [47]:
# %load tools/feature_format
#!/usr/bin/python

""" 
    A general tool for converting data from the
    dictionary format to an (n x k) python list that's 
    ready for training an sklearn algorithm

    n--no. of key-value pairs in dictonary
    k--no. of features being extracted

    dictionary keys are names of persons in dataset
    dictionary values are dictionaries, where each
        key-value pair in the dict is the name
        of a feature, and its value for that person

    In addition to converting a dictionary to a numpy 
    array, you may want to separate the labels from the
    features--this is what targetFeatureSplit is for

    so, if you want to have the poi label as the target,
    and the features you want to use are the person's
    salary and bonus, here's what you would do:

    feature_list = ["poi", "salary", "bonus"] 
    data_array = featureFormat( data_dictionary, feature_list )
    label, features = targetFeatureSplit(data_array)

    the line above (targetFeatureSplit) assumes that the
    label is the _first_ item in feature_list--very important
    that poi is listed first!
"""


import numpy as np

def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True, remove_any_zeroes=False, sort_keys = False):
    """ convert dictionary to numpy array of features
        remove_NaN = True will convert "NaN" string to 0.0
        remove_all_zeroes = True will omit any data points for which
            all the features you seek are 0.0
        remove_any_zeroes = True will omit any data points for which
            any of the features you seek are 0.0
        sort_keys = True sorts keys by alphabetical order. Setting the value as
            a string opens the corresponding pickle file with a preset key
            order (this is used for Python 3 compatibility, and sort_keys
            should be left as False for the course mini-projects).
        NOTE: first feature is assumed to be 'poi' and is not checked for
            removal for zero or missing values.
    """


    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print "error: key ", feature, " not present"
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)


def targetFeatureSplit( data ):
    """ 
        given a numpy array like the one returned from
        featureFormat, separate out the first feature
        and put it into its own list (this should be the 
        quantity you want to predict)

        return targets and features as separate lists

        (sklearn can generally handle both lists and numpy arrays as 
        input formats when training/predicting)
    """

    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )

    return target, features

In [41]:
feature_list = ["poi", "salary", "bonus", "total_payments"] 
data_array = featureFormat( enron_data, feature_list )
label, features = targetFeatureSplit(data_array)

In [46]:
print(len(features))

125
